In [2]:
import sys
sys.path.append('/workspace/Documents')
# imports
import os, sys

# third party imports 
import numpy as np 

import pandas as pd
import random
import nibabel as nb
import cv2
import ast

import Diffusion_motion_field.Build_lists.Build_list as Build_list
import Diffusion_motion_field.functions_collection as ff
import Diffusion_motion_field.Data_processing as Data_processing

main_path = '/mnt/camca_NAS/4DCT/models'

In [3]:
# load patient list
# data_sheet = os.path.join(os.path.dirname(main_path),'Patient_lists/uc/patient_list_MVF_diffusion_train_test_filtered_at_least_10tf.xlsx')
data_sheet = os.path.join(os.path.dirname(main_path),'Patient_lists/mgh/patient_list_MVF_diffusion_train_test.xlsx')
b = Build_list.Build(data_sheet)
patient_class_list, patient_id_list,_ = b.__build__(batch_list = [5])
# patient_class_list = patient_class_list[-20:]
# patient_id_list = patient_id_list[-20:]

In [4]:
# define study name
study_name = 'MVF_EDM_down_10tf_imgcon_EFcon_warp_orires'
epoch = 1930

Pull3 case 139

In [5]:
wl = -121
ww = 1155
for i in range(0,len(patient_id_list)): 
    patient_id = patient_id_list[i]
    patient_class = patient_class_list[i]
    if patient_id != 'case_139':
        continue
 
    print('patient_id:', patient_id, 'patient_class:', patient_class)

    timeframe_info = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_final_selection_timeframes.xlsx')
    row = timeframe_info[timeframe_info['patient_id'] == patient_id]
    sampled_time_frame_list = ast.literal_eval(row['sampled_time_frame_list'].iloc[0])
    normalized_time_frame_list = ast.literal_eval(row['normalized_time_frame_list_copy'].iloc[0])

    picked_tf_normalized = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]   
    print('picked tf:' ,picked_tf)

    folders = ff.find_all_target_files(['epoch'+str(epoch)+'_*'], os.path.join(main_path, study_name, 'pred_mvf_showcase', patient_class, patient_id))
    template_image = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class,patient_id,'img-nii-resampled-1.5mm/0.nii.gz')
    original_shape = nb.load(template_image).get_fdata().shape
    template_seg = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/predicted_seg/',patient_class,patient_id,'pred_s_0.nii.gz')
    seg = nb.load(template_seg).get_fdata(); seg = np.round(seg)
    LV_slice = [i for i in range(0,seg.shape[2]) if np.sum(seg[:,:,i]==1) > 0]

    slice_selected = np.round(np.linspace(LV_slice[0]+5, LV_slice[-1]-5, 6)).astype(int)
    print('slice_selected:', slice_selected)
    
    # load the mvf
    for f in range(0,len(folders)):
        print('loading mvf from:', folders[f])
        warped_imgs = np.zeros((len(picked_tf), original_shape[0], original_shape[1], original_shape[2]))

        for tf_i in range(0,len(picked_tf)):
            timeframe = picked_tf[tf_i]
            img_tf = nb.load(os.path.join(folders[f], 'warped_img_pred_tf'+str(timeframe)+'.nii.gz')).get_fdata()
            img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
            warped_imgs[tf_i,:,:,:] = ff.set_window(img_tf, wl, ww)

        print('warped_imgs shape:', warped_imgs.shape)
           
        # movies
        movie_folder = os.path.join(folders[f], 'movies')
        ff.make_folder([movie_folder])

        # per slice movie
        for slice_index in slice_selected:
            img = warped_imgs[:,:,:,slice_index]
            images = img * 255
            images = images.astype('uint8')
            height, width = images.shape[1], images.shape[2]
            fps = 10  
            output_file = os.path.join(folders[f],'movies', 'slice_'+str(slice_index)+'.mp4')

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, fps, (width, height), isColor=False)

            for i in range(images.shape[0]):
                frame = np.flip(images[i].T,0)
                out.write(frame)
            out.release()

        # combined
        # 2x3 panel
        img_combined = np.zeros((warped_imgs.shape[0], original_shape[0]*2, original_shape[1]*3))
        for tf_i in range(0, warped_imgs.shape[0]):
            img_combined[tf_i,:original_shape[0],:original_shape[1]] = np.flip(warped_imgs[tf_i,:,:,slice_selected[0]].T,0)
            img_combined[tf_i,:original_shape[0],original_shape[1]:original_shape[1]*2] = np.flip(warped_imgs[tf_i,:,:,slice_selected[1]].T,0)
            img_combined[tf_i,:original_shape[0],original_shape[1]*2:] = np.flip(warped_imgs[tf_i,:,:,slice_selected[2]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,:original_shape[1]] = np.flip(warped_imgs[tf_i,:,:,slice_selected[3]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,original_shape[1]:original_shape[1]*2] = np.flip(warped_imgs[tf_i,:,:,slice_selected[4]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,original_shape[1]*2:] = np.flip(warped_imgs[tf_i,:,:,slice_selected[5]].T,0)
        images_combined = img_combined * 255
        images_combined = images_combined.astype('uint8')
        height, width = images_combined.shape[1], images_combined.shape[2]
        fps = 10
        output_file_combined = os.path.join(folders[f],'movies', 'combined.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out_combined = cv2.VideoWriter(output_file_combined, fourcc, fps, (width, height), isColor=False)
        for i in range(images_combined.shape[0]):
            frame = images_combined[i]
            out_combined.write(frame)
        out_combined.release()
            

patient_id: case_139 patient_class: Pull_3
picked tf: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
slice_selected: [16 23 29 36 42 49]
loading mvf from: /mnt/camca_NAS/4DCT/models/MVF_EDM_down_10tf_imgcon_EFcon_warp_orires/pred_mvf_showcase/Pull_3/case_139/epoch1930_0
warped_imgs shape: (10, 111, 111, 73)
loading mvf from: /mnt/camca_NAS/4DCT/models/MVF_EDM_down_10tf_imgcon_EFcon_warp_orires/pred_mvf_showcase/Pull_3/case_139/epoch1930_1
warped_imgs shape: (10, 111, 111, 73)
loading mvf from: /mnt/camca_NAS/4DCT/models/MVF_EDM_down_10tf_imgcon_EFcon_warp_orires/pred_mvf_showcase/Pull_3/case_139/epoch1930_2
warped_imgs shape: (10, 111, 111, 73)
loading mvf from: /mnt/camca_NAS/4DCT/models/MVF_EDM_down_10tf_imgcon_EFcon_warp_orires/pred_mvf_showcase/Pull_3/case_139/epoch1930_3
warped_imgs shape: (10, 111, 111, 73)


In [4]:
# reference
wl = -121
ww = 1155
for i in range(0,len(patient_id_list)): 
    patient_id = patient_id_list[i]
    patient_class = patient_class_list[i]
    if patient_id != 'case_139':
        continue
 
    print('patient_id:', patient_id, 'patient_class:', patient_class)

    timeframe_info = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_final_selection_timeframes.xlsx')
    row = timeframe_info[timeframe_info['patient_id'] == patient_id]
    sampled_time_frame_list = ast.literal_eval(row['sampled_time_frame_list'].iloc[0])
    normalized_time_frame_list = ast.literal_eval(row['normalized_time_frame_list_copy'].iloc[0])

    picked_tf_normalized = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]   
    print('picked tf:' ,picked_tf)

    folders = [os.path.join('/mnt/camca_NAS/4DCT/reference/',patient_class,patient_id,'image_warped_mvf_original'),os.path.join('/mnt/camca_NAS/4DCT/reference/',patient_class,patient_id,'image_original')]
    template_image = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class,patient_id,'img-nii-resampled-1.5mm/0.nii.gz')
    original_shape = nb.load(template_image).get_fdata().shape
    template_seg = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/predicted_seg/',patient_class,patient_id,'pred_s_0.nii.gz')
    seg = nb.load(template_seg).get_fdata(); seg = np.round(seg)
    LV_slice = [i for i in range(0,seg.shape[2]) if np.sum(seg[:,:,i]==1) > 0]

    slice_selected = np.round(np.linspace(LV_slice[0]+5, LV_slice[-1]-5, 6)).astype(int)
    print('slice_selected:', slice_selected)
    
    # load the mvf
    for f in range(0,len(folders)):
        warped_imgs = np.zeros((len(picked_tf), original_shape[0], original_shape[1], original_shape[2]))

        for tf_i in range(0,len(picked_tf)):
            timeframe = picked_tf[tf_i]
            img_tf = nb.load(os.path.join(folders[f], str(timeframe)+'.nii.gz')).get_fdata()
            img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
            warped_imgs[tf_i,:,:,:] = ff.set_window(img_tf, wl, ww)
           
        # movies
        folder = folders[f]
        movie_folder = os.path.join(folder, 'movies')
        ff.make_folder([movie_folder])
        slice_indexs = [20,30,40,50,60]
        for slice_index in slice_selected:
            img = warped_imgs[:,:,:,slice_index]
            images = img * 255
            images = images.astype('uint8')
            height, width = images.shape[1], images.shape[2]
            fps = 10  
            output_file = os.path.join(folder,'movies', 'slice_'+str(slice_index)+'.mp4')

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, fps, (width, height), isColor=False)

            for i in range(images.shape[0]):
                frame = np.flip(images[i].T,0)
                out.write(frame)
            out.release()

        # combined
        # 2x3 panel
        img_combined = np.zeros((warped_imgs.shape[0], original_shape[0]*2, original_shape[1]*3))
        for tf_i in range(0, warped_imgs.shape[0]):
            img_combined[tf_i,:original_shape[0],:original_shape[1]] = np.flip(warped_imgs[tf_i,:,:,slice_selected[0]].T,0)
            img_combined[tf_i,:original_shape[0],original_shape[1]:original_shape[1]*2] = np.flip(warped_imgs[tf_i,:,:,slice_selected[1]].T,0)
            img_combined[tf_i,:original_shape[0],original_shape[1]*2:] = np.flip(warped_imgs[tf_i,:,:,slice_selected[2]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,:original_shape[1]] = np.flip(warped_imgs[tf_i,:,:,slice_selected[3]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,original_shape[1]:original_shape[1]*2] = np.flip(warped_imgs[tf_i,:,:,slice_selected[4]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,original_shape[1]*2:] = np.flip(warped_imgs[tf_i,:,:,slice_selected[5]].T,0)
        images_combined = img_combined * 255
        images_combined = images_combined.astype('uint8')
        height, width = images_combined.shape[1], images_combined.shape[2]
        fps = 10
        output_file_combined = os.path.join(folders[f],'movies', 'combined.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out_combined = cv2.VideoWriter(output_file_combined, fourcc, fps, (width, height), isColor=False)
        for i in range(images_combined.shape[0]):
            frame = images_combined[i]
            out_combined.write(frame)
        out_combined.release()

patient_id: case_139 patient_class: Pull_3
picked tf: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
slice_selected: [16 23 29 36 42 49]


In [6]:
### image illustration
patient_class = 'Pull_3'
patient_id = 'case_139'
save_folder = os.path.join('/mnt/camca_NAS/4DCT/pictures/manuscript/4DCT_examples/',patient_class, patient_id)
ff.make_folder([os.path.dirname(save_folder), save_folder])
time_frame_list = [0,2,4,6,8,10,12,14,16,18]
wl = -121
ww = 1155
slice_n = 36

template_image = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class,patient_id,'img-nii-resampled-1.5mm/0.nii.gz')
original_shape = nb.load(template_image).get_fdata().shape

# reference
image_folder = os.path.join('/mnt/camca_NAS/4DCT/reference/',patient_class,patient_id,'image_warped_mvf_original')
for tf in time_frame_list:
    img_tf = nb.load(os.path.join(image_folder, str(tf)+'.nii.gz')).get_fdata()
    img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
    img_tf = img_tf[:,:,slice_n]
    ff.save_grayscale_image(np.flip(img_tf.T,axis = 0),os.path.join(save_folder,'reference_tf_'+str(tf)+'.png'), WL=wl, WW=ww)

# factual synthesis
image_folder = os.path.join(main_path, study_name, 'pred_mvf_showcase', patient_class, patient_id,'epoch1930_0')
for tf in time_frame_list:
    img_tf = nb.load(os.path.join(image_folder, 'warped_img_pred_tf'+str(tf)+'.nii.gz')).get_fdata()
    img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
    img_tf = img_tf[:,:,slice_n]
    ff.save_grayscale_image(np.flip(img_tf.T,axis = 0),os.path.join(save_folder,'factual_synthesis_tf_'+str(tf)+'.png'), WL=wl, WW=ww)

# counterfactual synthesis
image_folder = os.path.join(main_path, study_name, 'pred_mvf_showcase', patient_class, patient_id,'epoch1930_1')
for tf in time_frame_list:
    img_tf = nb.load(os.path.join(image_folder, 'warped_img_pred_tf'+str(tf)+'.nii.gz')).get_fdata()
    img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
    img_tf = img_tf[:,:,slice_n]
    ff.save_grayscale_image(np.flip(img_tf.T,axis = 0),os.path.join(save_folder,'counterfactual_synthesis_tf_'+str(tf)+'.png'), WL=wl, WW=ww)



Pull 4 case 162

In [ ]:
wl = -50
ww = 1000
for i in range(0,len(patient_id_list)): 
    patient_id = patient_id_list[i]
    patient_class = patient_class_list[i]
    if patient_id != 'case_162':
        continue
 
    print('patient_id:', patient_id, 'patient_class:', patient_class)

    timeframe_info = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_final_selection_timeframes.xlsx')
    row = timeframe_info[timeframe_info['patient_id'] == patient_id]
    sampled_time_frame_list = ast.literal_eval(row['sampled_time_frame_list'].iloc[0])
    normalized_time_frame_list = ast.literal_eval(row['normalized_time_frame_list_copy'].iloc[0])

    picked_tf_normalized = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]   
    print('picked tf:' ,picked_tf)

    folders = ff.find_all_target_files(['epoch'+str(epoch)+'_*'], os.path.join(main_path, study_name, 'pred_mvf_showcase', patient_class, patient_id))
    template_image = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class,patient_id,'img-nii-resampled-1.5mm/0.nii.gz')
    original_shape = nb.load(template_image).get_fdata().shape
    template_seg = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/predicted_seg/',patient_class,patient_id,'pred_s_0.nii.gz')
    seg = nb.load(template_seg).get_fdata(); seg = np.round(seg)
    LV_slice = [i for i in range(0,seg.shape[2]) if np.sum(seg[:,:,i]==1) > 0]

    slice_selected = np.round(np.linspace(LV_slice[0]+5, LV_slice[-1]-5, 6)).astype(int)
    print('slice_selected:', slice_selected)
    
    # load the mvf
    for f in range(0,len(folders)):
        print('loading mvf from:', folders[f])
        warped_imgs = np.zeros((len(picked_tf), original_shape[0], original_shape[1], original_shape[2]))

        for tf_i in range(0,len(picked_tf)):
            timeframe = picked_tf[tf_i]
            img_tf = nb.load(os.path.join(folders[f], 'warped_img_pred_tf'+str(timeframe)+'.nii.gz')).get_fdata()
            img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
            warped_imgs[tf_i,:,:,:] = ff.set_window(img_tf, wl, ww)

        print('warped_imgs shape:', warped_imgs.shape)
           
        # movies
        movie_folder = os.path.join(folders[f], 'movies')
        ff.make_folder([movie_folder])

        # per slice movie
        for slice_index in slice_selected:
            img = warped_imgs[:,:,:,slice_index]
            images = img * 255
            images = images.astype('uint8')
            height, width = images.shape[1], images.shape[2]
            fps = 10  
            output_file = os.path.join(folders[f],'movies', 'slice_'+str(slice_index)+'.mp4')

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, fps, (width, height), isColor=False)

            for i in range(images.shape[0]):
                frame = np.flip(images[i].T,0)
                out.write(frame)
            out.release()

        # combined
        # 2x3 panel
        img_combined = np.zeros((warped_imgs.shape[0], original_shape[0]*2, original_shape[1]*3))
        for tf_i in range(0, warped_imgs.shape[0]):
            img_combined[tf_i,:original_shape[0],:original_shape[1]] = np.flip(warped_imgs[tf_i,:,:,slice_selected[0]].T,0)
            img_combined[tf_i,:original_shape[0],original_shape[1]:original_shape[1]*2] = np.flip(warped_imgs[tf_i,:,:,slice_selected[1]].T,0)
            img_combined[tf_i,:original_shape[0],original_shape[1]*2:] = np.flip(warped_imgs[tf_i,:,:,slice_selected[2]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,:original_shape[1]] = np.flip(warped_imgs[tf_i,:,:,slice_selected[3]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,original_shape[1]:original_shape[1]*2] = np.flip(warped_imgs[tf_i,:,:,slice_selected[4]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,original_shape[1]*2:] = np.flip(warped_imgs[tf_i,:,:,slice_selected[5]].T,0)
        images_combined = img_combined * 255
        images_combined = images_combined.astype('uint8')
        height, width = images_combined.shape[1], images_combined.shape[2]
        fps = 10
        output_file_combined = os.path.join(folders[f],'movies', 'combined.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out_combined = cv2.VideoWriter(output_file_combined, fourcc, fps, (width, height), isColor=False)
        for i in range(images_combined.shape[0]):
            frame = images_combined[i]
            out_combined.write(frame)
        out_combined.release()
            

In [ ]:
# reference
wl = -50
ww = 1000
for i in range(0,len(patient_id_list)): 
    patient_id = patient_id_list[i]
    patient_class = patient_class_list[i]
    if patient_id != 'case_162':
        continue
 
    print('patient_id:', patient_id, 'patient_class:', patient_class)

    timeframe_info = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_final_selection_timeframes.xlsx')
    row = timeframe_info[timeframe_info['patient_id'] == patient_id]
    sampled_time_frame_list = ast.literal_eval(row['sampled_time_frame_list'].iloc[0])
    normalized_time_frame_list = ast.literal_eval(row['normalized_time_frame_list_copy'].iloc[0])

    picked_tf_normalized = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]   
    print('picked tf:' ,picked_tf)

    folders = [os.path.join('/mnt/camca_NAS/4DCT/reference/',patient_class,patient_id,'image_warped_mvf_original'),os.path.join('/mnt/camca_NAS/4DCT/reference/',patient_class,patient_id,'image_original')]
    template_image = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class,patient_id,'img-nii-resampled-1.5mm/0.nii.gz')
    original_shape = nb.load(template_image).get_fdata().shape
    template_seg = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/predicted_seg/',patient_class,patient_id,'pred_s_0.nii.gz')
    seg = nb.load(template_seg).get_fdata(); seg = np.round(seg)
    LV_slice = [i for i in range(0,seg.shape[2]) if np.sum(seg[:,:,i]==1) > 0]

    slice_selected = np.round(np.linspace(LV_slice[0]+5, LV_slice[-1]-5, 6)).astype(int)
    print('slice_selected:', slice_selected)
    
    # load the mvf
    for f in range(0,len(folders)):
        warped_imgs = np.zeros((len(picked_tf), original_shape[0], original_shape[1], original_shape[2]))

        for tf_i in range(0,len(picked_tf)):
            timeframe = picked_tf[tf_i]
            img_tf = nb.load(os.path.join(folders[f], str(timeframe)+'.nii.gz')).get_fdata()
            img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
            warped_imgs[tf_i,:,:,:] = ff.set_window(img_tf, wl, ww)
           
        # movies
        folder = folders[f]
        movie_folder = os.path.join(folder, 'movies')
        ff.make_folder([movie_folder])
        slice_indexs = [20,30,40,50,60]
        for slice_index in slice_selected:
            img = warped_imgs[:,:,:,slice_index]
            images = img * 255
            images = images.astype('uint8')
            height, width = images.shape[1], images.shape[2]
            fps = 10  
            output_file = os.path.join(folder,'movies', 'slice_'+str(slice_index)+'.mp4')

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, fps, (width, height), isColor=False)

            for i in range(images.shape[0]):
                frame = np.flip(images[i].T,0)
                out.write(frame)
            out.release()

        # combined
        # 2x3 panel
        img_combined = np.zeros((warped_imgs.shape[0], original_shape[0]*2, original_shape[1]*3))
        for tf_i in range(0, warped_imgs.shape[0]):
            img_combined[tf_i,:original_shape[0],:original_shape[1]] = np.flip(warped_imgs[tf_i,:,:,slice_selected[0]].T,0)
            img_combined[tf_i,:original_shape[0],original_shape[1]:original_shape[1]*2] = np.flip(warped_imgs[tf_i,:,:,slice_selected[1]].T,0)
            img_combined[tf_i,:original_shape[0],original_shape[1]*2:] = np.flip(warped_imgs[tf_i,:,:,slice_selected[2]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,:original_shape[1]] = np.flip(warped_imgs[tf_i,:,:,slice_selected[3]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,original_shape[1]:original_shape[1]*2] = np.flip(warped_imgs[tf_i,:,:,slice_selected[4]].T,0)
            img_combined[tf_i,original_shape[0]:original_shape[0]*2,original_shape[1]*2:] = np.flip(warped_imgs[tf_i,:,:,slice_selected[5]].T,0)
        images_combined = img_combined * 255
        images_combined = images_combined.astype('uint8')
        height, width = images_combined.shape[1], images_combined.shape[2]
        fps = 10
        output_file_combined = os.path.join(folders[f],'movies', 'combined.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out_combined = cv2.VideoWriter(output_file_combined, fourcc, fps, (width, height), isColor=False)
        for i in range(images_combined.shape[0]):
            frame = images_combined[i]
            out_combined.write(frame)
        out_combined.release()

In [8]:
### image illustration
patient_class = 'Pull_4'
patient_id = 'case_162'
save_folder = os.path.join('/mnt/camca_NAS/4DCT/pictures/manuscript/4DCT_examples/',patient_class, patient_id)
ff.make_folder([os.path.dirname(save_folder), save_folder])
time_frame_list = [0,2,4,6,8,10,12,14,16,18]
wl = -50
ww = 1000
slice_n = 37

template_image = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class,patient_id,'img-nii-resampled-1.5mm/0.nii.gz')
original_shape = nb.load(template_image).get_fdata().shape

# reference
image_folder = os.path.join('/mnt/camca_NAS/4DCT/reference/',patient_class,patient_id,'image_warped_mvf_original')
for tf in time_frame_list:
    img_tf = nb.load(os.path.join(image_folder, str(tf)+'.nii.gz')).get_fdata()
    img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
    img_tf = img_tf[:,:,slice_n]
    ff.save_grayscale_image(np.flip(img_tf.T,axis = 0),os.path.join(save_folder,'reference_tf_'+str(tf)+'.png'), WL=wl, WW=ww)

# factual synthesis
image_folder = os.path.join(main_path, study_name, 'pred_mvf_showcase', patient_class, patient_id,'epoch1930_0')
for tf in time_frame_list:
    img_tf = nb.load(os.path.join(image_folder, 'warped_img_pred_tf'+str(tf)+'.nii.gz')).get_fdata()
    img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
    img_tf = img_tf[:,:,slice_n]
    ff.save_grayscale_image(np.flip(img_tf.T,axis = 0),os.path.join(save_folder,'factual_synthesis_tf_'+str(tf)+'.png'), WL=wl, WW=ww)

# counterfactual synthesis
image_folder = os.path.join(main_path, study_name, 'pred_mvf_showcase', patient_class, patient_id,'epoch1930_3')
for tf in time_frame_list:
    img_tf = nb.load(os.path.join(image_folder, 'warped_img_pred_tf'+str(tf)+'.nii.gz')).get_fdata()
    img_tf = Data_processing.correct_shift_caused_in_pad_crop_loop(Data_processing.crop_or_pad(img_tf, [original_shape[0], original_shape[1], original_shape[2]], value = np.min(img_tf)))
    img_tf = img_tf[:,:,slice_n]
    ff.save_grayscale_image(np.flip(img_tf.T,axis = 0),os.path.join(save_folder,'counterfactual_synthesis_tf_'+str(tf)+'.png'), WL=wl, WW=ww)

